In [8]:
mkdir dataset


In [11]:
cd dataset

/content/dataset


In [13]:
mv test validation

In [15]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import os

class FaceEmotionDetector:
    def __init__(self):
        # Define emotion labels
        self.emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        self.img_size = 48  # FER dataset uses 48x48 grayscale images
        self.model = None
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def build_model(self):
        """Build the CNN model architecture"""
        model = Sequential()

        # First convolution block
        model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(self.img_size, self.img_size, 1)))
        model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Second convolution block
        model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Third convolution block
        model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Fully connected layers
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(len(self.emotions), activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=0.0005),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model
        return model

    def train(self, train_dir, validation_dir, epochs=50, batch_size=64):
        """Train the model using data from the given directories"""
        if self.model is None:
            self.build_model()

        # Create data generators for training and validation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        validation_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical'
        )

        validation_generator = validation_datagen.flow_from_directory(
            validation_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical'
        )

        # Set up callbacks
        checkpoint = ModelCheckpoint(
            "emotion_model_best.h5",
            monitor='val_accuracy',
            verbose=1,
            save_best_only=True,
            mode='max'
        )

        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            verbose=1,
            restore_best_weights=True
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            verbose=1,
            min_lr=0.00001
        )

        callbacks = [checkpoint, early_stopping, reduce_lr]

        # Train the model
        history = self.model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // batch_size,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // batch_size,
            callbacks=callbacks
        )

        # Save the model
        self.model.save('emotion_model.h5')

        return history

    def load_trained_model(self, model_path):
        """Load a trained model from disk"""
        self.model = load_model(model_path)

    def detect_emotion(self, image):
        """
        Detect faces in the image and predict their emotions

        Args:
            image: numpy array of the image (BGR format from OpenCV)

        Returns:
            List of tuples with (face_box, emotion, confidence)
        """
        if self.model is None:
            raise ValueError("Model not trained or loaded")

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        result = []
        for (x, y, w, h) in faces:
            # Extract the face ROI
            roi_gray = gray[y:y+h, x:x+h]

            # Resize to expected size
            roi_gray = cv2.resize(roi_gray, (self.img_size, self.img_size))

            # Normalize and reshape for model input
            roi = roi_gray.astype('float') / 255.0
            roi = np.expand_dims(roi, axis=0)
            roi = np.expand_dims(roi, axis=-1)

            # Make prediction
            prediction = self.model.predict(roi, verbose=0)[0]

            # Get max confidence emotion
            emotion_idx = np.argmax(prediction)
            emotion = self.emotions[emotion_idx]
            confidence = prediction[emotion_idx]

            result.append(((x, y, w, h), emotion, confidence))

        return result

    def visualize_results(self, image, results):
        """
        Draw the results on the image

        Args:
            image: Original image
            results: List of (face_box, emotion, confidence) tuples

        Returns:
            Image with annotated faces and emotions
        """
        output = image.copy()

        for (x, y, w, h), emotion, confidence in results:
            # Draw rectangle around face
            cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Add text with emotion and confidence
            text = f"{emotion}: {confidence:.2f}"
            y_offset = y - 10 if y - 10 > 10 else y + h + 10
            cv2.putText(output, text, (x, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        return output


def demo_with_webcam():
    """Run a demo using webcam feed"""
    detector = FaceEmotionDetector()

    # Load pre-trained model (assuming it exists)
    try:
        detector.load_trained_model('emotion_model_best.h5')
        print("Model loaded successfully")
    except:
        print("Pre-trained model not found. Please train the model first.")
        return

    # Open webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Could not open webcam")
        return

    print("Press 'q' to quit")

    while True:
        # Read frame
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Detect emotions
        results = detector.detect_emotion(frame)

        # Visualize results
        output_frame = detector.visualize_results(frame, results)

        # Display result
        cv2.imshow('Emotion Detection', output_frame)

        # Break on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


def download_fer_dataset():
    """
    Instructions to download and prepare the FER2013 dataset

    The FER2013 dataset is commonly used for facial emotion recognition
    and contains ~35K 48x48 grayscale images of faces with 7 emotion categories.
    """
    print("To train this model, you need the FER2013 dataset.")
    print("You can download it from Kaggle: https://www.kaggle.com/datasets/msambare/fer2013")
    print("\nAfter downloading, organize the dataset as follows:")
    print("dataset/")
    print("├── train/")
    print("│   ├── angry/")
    print("│   ├── disgust/")
    print("│   ├── fear/")
    print("│   ├── happy/")
    print("│   ├── sad/")
    print("│   ├── surprise/")
    print("│   └── neutral/")
    print("└── validation/")
    print("    ├── angry/")
    print("    ├── disgust/")
    print("    ├── fear/")
    print("    ├── happy/")
    print("    ├── sad/")
    print("    ├── surprise/")
    print("    └── neutral/")
    print("\nThen you can train the model using:")
    print("detector = FaceEmotionDetector()")
    print("detector.train('dataset/train', 'dataset/validation')")


if __name__ == "__main__":
    # Choose what to do
    print("1. Download and prepare dataset")
    print("2. Run webcam demo")
    choice = input("Enter your choice (1/2): ")

    if choice == '1':
        download_fer_dataset()
    elif choice == '2':
        demo_with_webcam()
    else:
        print("Invalid choice")

1. Download and prepare dataset
2. Run webcam demo
Enter your choice (1/2): 1
To train this model, you need the FER2013 dataset.
You can download it from Kaggle: https://www.kaggle.com/datasets/msambare/fer2013

After downloading, organize the dataset as follows:
dataset/
├── train/
│   ├── angry/
│   ├── disgust/
│   ├── fear/
│   ├── happy/
│   ├── sad/
│   ├── surprise/
│   └── neutral/
└── validation/
    ├── angry/
    ├── disgust/
    ├── fear/
    ├── happy/
    ├── sad/
    ├── surprise/
    └── neutral/

Then you can train the model using:
detector = FaceEmotionDetector()
detector.train('dataset/train', 'dataset/validation')


In [16]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import os

class LightweightFaceEmotionDetector:
    def __init__(self):
        # Define emotion labels
        self.emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        self.img_size = 48  # FER dataset uses 48x48 grayscale images
        self.model = None

    def build_model(self):
        """Build the lightweight CNN model architecture"""
        model = Sequential()

        # First convolution block
        model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(self.img_size, self.img_size, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Second convolution block
        model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Fully connected layers
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(len(self.emotions), activation='softmax'))

        # Compile the model
        model.compile(
            optimizer=Adam(learning_rate=0.0005),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model
        return model

    def train(self, train_dir, validation_dir, epochs=50, batch_size=64):
        """Train the model using data from the given directories"""
        if self.model is None:
            self.build_model()

        # Create data generators for training and validation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        validation_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical'
        )

        validation_generator = validation_datagen.flow_from_directory(
            validation_dir,
            target_size=(self.img_size, self.img_size),
            batch_size=batch_size,
            color_mode='grayscale',
            class_mode='categorical'
        )

        # Set up callbacks
        checkpoint = ModelCheckpoint(
            "lightweight_emotion_model_best.h5",
            monitor='val_accuracy',
            verbose=1,
            save_best_only=True,
            mode='max'
        )

        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            verbose=1,
            restore_best_weights=True
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            verbose=1,
            min_lr=0.00001
        )

        callbacks = [checkpoint, early_stopping, reduce_lr]

        # Train the model
        history = self.model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // batch_size,
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.samples // batch_size,
            callbacks=callbacks
        )

        # Save the model
        self.model.save('lightweight_emotion_model.h5')

        return history

In [19]:
# Paths to your dataset
train_dir = 'train'
validation_dir = 'validation'

# Initialize the detector
detector = LightweightFaceEmotionDetector()

# Train the model
history = detector.train(train_dir, validation_dir, epochs=50, batch_size=64)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.2019 - loss: 2.3908
Epoch 1: val_accuracy improved from -inf to 0.18025, saving model to lightweight_emotion_model_best.h5


448/448 ━━━━━━━━━━━━━━━━━━━━ 183s 399ms/step - accuracy: 0.2020 - loss: 2.3904 - val_accuracy: 0.1802 - val_loss: 8.8572 - learning_rate: 5.0000e-04
Epoch 2/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:43 365ms/step - accuracy: 0.2500 - loss: 2.0104

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

448/448 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.5312 - loss: 1.2577 - val_accuracy: 0.5208 - val_loss: 1.2540 - learning_rate: 1.0000e-04
Epoch 29/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.4631 - loss: 1.4042
Epoch 29: val_accuracy did not improve from 0.52079
448/448 ━━━━━━━━━━━━━━━━━━━━ 166s 371ms/step - accuracy: 0.4631 - loss: 1.4042 - val_accuracy: 0.5075 - val_loss: 1.2791 - learning_rate: 1.0000e-04
Epoch 30/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:02 275ms/step - accuracy: 0.3125 - loss: 1.7100
Epoch 30: val_accuracy did not improve from 0.52079
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.3125 - loss: 1.7100 - val_accuracy: 0.5075 - val_loss: 1.2788 - learning_rate: 1.0000e-04
Epoch 31/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.4648 - loss: 1.4057
Epoch 31: val_accuracy did not improve from 0.52079
448/448 ━━━━━━━━━━━━━━━━━━━━ 201s 448ms/step - accuracy: 0.4648 - loss: 1.4058 - val_accuracy: 0.5152 - val_loss: 1.2753 - learning_

448/448 ━━━━━━━━━━━━━━━━━━━━ 218s 372ms/step - accuracy: 0.4693 - loss: 1.3963 - val_accuracy: 0.5244 - val_loss: 1.2508 - learning_rate: 2.0000e-05
Epoch 36/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:03 276ms/step - accuracy: 0.5156 - loss: 1.3256
Epoch 36: val_accuracy improved from 0.52441 to 0.52455, saving model to lightweight_emotion_model_best.h5


448/448 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.5156 - loss: 1.3256 - val_accuracy: 0.5246 - val_loss: 1.2506 - learning_rate: 2.0000e-05
Epoch 37/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.4674 - loss: 1.3950
Epoch 37: val_accuracy did not improve from 0.52455
448/448 ━━━━━━━━━━━━━━━━━━━━ 166s 371ms/step - accuracy: 0.4675 - loss: 1.3950 - val_accuracy: 0.5246 - val_loss: 1.2499 - learning_rate: 2.0000e-05
Epoch 38/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:12 297ms/step - accuracy: 0.4062 - loss: 1.6421
Epoch 38: val_accuracy did not improve from 0.52455
448/448 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.4062 - loss: 1.6421 - val_accuracy: 0.5246 - val_loss: 1.2504 - learning_rate: 2.0000e-05
Epoch 39/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.4737 - loss: 1.3881
Epoch 39: val_accuracy improved from 0.52455 to 0.52637, saving model to lightweight_emotion_model_best.h5


448/448 ━━━━━━━━━━━━━━━━━━━━ 192s 368ms/step - accuracy: 0.4737 - loss: 1.3881 - val_accuracy: 0.5264 - val_loss: 1.2425 - learning_rate: 2.0000e-05
Epoch 40/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:35 482ms/step - accuracy: 0.4844 - loss: 1.3739
Epoch 40: val_accuracy improved from 0.52637 to 0.52734, saving model to lightweight_emotion_model_best.h5


448/448 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.4844 - loss: 1.3739 - val_accuracy: 0.5273 - val_loss: 1.2410 - learning_rate: 2.0000e-05
Epoch 41/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.4681 - loss: 1.3950
Epoch 41: val_accuracy did not improve from 0.52734
448/448 ━━━━━━━━━━━━━━━━━━━━ 167s 373ms/step - accuracy: 0.4681 - loss: 1.3950 - val_accuracy: 0.5236 - val_loss: 1.2558 - learning_rate: 2.0000e-05
Epoch 42/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:03 277ms/step - accuracy: 0.4375 - loss: 1.4054
Epoch 42: val_accuracy did not improve from 0.52734
448/448 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.4375 - loss: 1.4054 - val_accuracy: 0.5236 - val_loss: 1.2565 - learning_rate: 2.0000e-05
Epoch 43/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.4726 - loss: 1.3865
Epoch 43: val_accuracy did not improve from 0.52734
448/448 ━━━━━━━━━━━━━━━━━━━━ 171s 382ms/step - accuracy: 0.4726 - loss: 1.3865 - val_accuracy: 0.5239 - val_loss: 1.2510 - learning_ra

In [26]:
def demo_with_webcam():
    """Run a demo using webcam feed"""
    detector = LightweightFaceEmotionDetector()

    # Load pre-trained model (assuming it exists)
    try:
        detector.load_trained_model('lightweight_emotion_model_best.h5')
        print("Model loaded successfully")
    except:
        print("Pre-trained model not found. Please train the model first.")
        return

    # Open webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Could not open webcam")
        return

    print("Press 'q' to quit")

    while True:
        # Read frame
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Detect emotions
        results = detector.detect_emotion(frame)

        # Visualize results
        output_frame = detector.visualize_results(frame, results)

        # Display result
        cv2.imshow('Emotion Detection', output_frame)

        # Break on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

In [27]:
demo_with_webcam()


Pre-trained model not found. Please train the model first.


In [23]:
ls


lightweight_emotion_model_best.h5  lightweight_emotion_model.h5  train/  validation/
